First download the .csvs 

!curl http://web.mta.info/developers/data/nyct/turnstile/turnstile_160903.txt > '9_3_2016_turnstiles.csv'  
!curl http://web.mta.info/developers/data/nyct/turnstile/turnstile_160910.txt > '9_10_2016_turnstiles.csv'  
!curl http://web.mta.info/developers/data/nyct/turnstile/turnstile_160917.txt > '9_17_2016_turnstiles.csv'  


Read in the .csvs and create one dataframe containing all of them.

In [39]:
import pandas as pd

sep_03_2016_df = pd.read_csv('9_3_2016_turnstiles.csv')
sep_10_2016_df = pd.read_csv('9_10_2016_turnstiles.csv')
sep_17_2016_df = pd.read_csv('9_17_2016_turnstiles.csv')


df = pd.concat([sep_03_2016_df,sep_10_2016_df,sep_17_2016_df],ignore_index=True)
# drop duplicates here? df.drop_duplicates()
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,00:00:00,REGULAR,5799442,1966041
1,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,04:00:00,REGULAR,5799463,1966044
2,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,08:00:00,REGULAR,5799492,1966079
3,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,12:00:00,REGULAR,5799610,1966155
4,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,16:00:00,REGULAR,5799833,1966214


Create 'STATID', 'TIMESTAMP', 'DIFFERENCE' columns.
STATID is a unique identifier for a station using the station's name and the train lines that can be accessed there.
TIMESTAMP is a combination of the DATE and TIME fields.
DIFFERENCE is a count of the number of entries that occured since the last recorded TIMESTAMP.

In [40]:
df['STATID'] = df.STATION + df.LINENAME

In [41]:
df['TIMESTAMP'] = pd.to_datetime((df.DATE + ' ' + df.TIME), format='%m/%d/%Y %H:%M:%S')

In [42]:
df = df.sort_values(['STATID', 'UNIT','SCP', 'TIMESTAMP'])
df['DIFFERENCE']=df.groupby(['STATID','UNIT','SCP'],as_index=False)['ENTRIES'].transform(pd.Series.diff)['ENTRIES']

In [43]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,STATID,TIMESTAMP,DIFFERENCE
29649,H007,R248,00-00-00,1 AV,L,BMT,08/27/2016,00:00:00,REGULAR,11448160,12524556,1 AVL,2016-08-27 00:00:00,NaN
29650,H007,R248,00-00-00,1 AV,L,BMT,08/27/2016,04:00:00,REGULAR,11448263,12524848,1 AVL,2016-08-27 04:00:00,103
29651,H007,R248,00-00-00,1 AV,L,BMT,08/27/2016,08:00:00,REGULAR,11448362,12525150,1 AVL,2016-08-27 08:00:00,99
29652,H007,R248,00-00-00,1 AV,L,BMT,08/27/2016,16:00:00,REGULAR,11449527,12526744,1 AVL,2016-08-27 16:00:00,1165
29653,H007,R248,00-00-00,1 AV,L,BMT,08/27/2016,20:00:00,REGULAR,11450095,12527835,1 AVL,2016-08-27 20:00:00,568


Now look for outliers in the DIFFERENCE data.

In [44]:
sorted_df = df.sort_values('DIFFERENCE')
pd.options.display.max_rows = 200
sorted_df.DIFFERENCE.head(100)

389638   -1894869117
64198    -1494568993
134614    -417796251
356534    -117440532
547404     -74671740
390615     -66849133
64324      -14127545
575628     -12497846
154406     -11078280
299021     -10561669
167305      -9168695
536927      -6130945
487838      -3664512
487338      -3252754
362071      -2337158
248714      -2042719
303973      -1460031
422724      -1054560
517543      -1014544
422766       -761957
248671       -703669
176387       -563964
112578       -533710
228432       -287991
164456       -238404
112658       -238193
422856       -227132
422904       -197042
308908       -193763
308906       -193670
382471       -174613
34970        -138983
228340       -138776
228519       -122268
228570        -98988
228477        -95614
568904        -87315
228255        -61491
228615        -44919
228388        -40061
503477        -36263
422814        -29042
228659        -25039
422542        -24360
308947        -21942
422587        -20866
110134        -14620
308670       

Without any more investigation it seem reasonable to remove the first 33 entries. 

In [45]:
sorted_df = sorted_df[sorted_df['DIFFERENCE'] > -122268]

In [46]:
sorted_df.DIFFERENCE.head(5)

228570   -98988
228477   -95614
568904   -87315
228255   -61491
228615   -44919
Name: DIFFERENCE, dtype: float64

In [47]:
sorted_df[sorted_df['DIFFERENCE'] == -98988]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,STATID,TIMESTAMP,DIFFERENCE
228570,JFK03,R536,00-03-02,JFK JAMAICA CT1,E,IND,09/08/2016,13:00:00,REGULAR,55,59,JFK JAMAICA CT1E,2016-09-08 13:00:00,-98988


In [48]:
sorted_df[sorted_df['DATE'] == '09/08/2016'][sorted_df['UNIT'] == 'R536'][sorted_df['SCP'] == '00-03-02'].sort_values(['STATID', 'UNIT','SCP', 'TIMESTAMP'])

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,STATID,TIMESTAMP,DIFFERENCE
228567,JFK03,R536,00-03-02,JFK JAMAICA CT1,E,IND,09/08/2016,01:00:00,REGULAR,98839,122559,JFK JAMAICA CT1E,2016-09-08 01:00:00,28
228568,JFK03,R536,00-03-02,JFK JAMAICA CT1,E,IND,09/08/2016,05:00:00,REGULAR,98875,122604,JFK JAMAICA CT1E,2016-09-08 05:00:00,36
228569,JFK03,R536,00-03-02,JFK JAMAICA CT1,E,IND,09/08/2016,09:00:00,REGULAR,99043,122763,JFK JAMAICA CT1E,2016-09-08 09:00:00,168
228570,JFK03,R536,00-03-02,JFK JAMAICA CT1,E,IND,09/08/2016,13:00:00,REGULAR,55,59,JFK JAMAICA CT1E,2016-09-08 13:00:00,-98988
228571,JFK03,R536,00-03-02,JFK JAMAICA CT1,E,IND,09/08/2016,17:00:00,REGULAR,372,401,JFK JAMAICA CT1E,2016-09-08 17:00:00,317
228572,JFK03,R536,00-03-02,JFK JAMAICA CT1,E,IND,09/08/2016,21:00:00,REGULAR,679,693,JFK JAMAICA CT1E,2016-09-08 21:00:00,307


Looks like counter restarted inbetween records. Should disregard count. 

In [49]:
sorted_df.DIFFERENCE.head(50)

228570   -98988
228477   -95614
568904   -87315
228255   -61491
228615   -44919
228388   -40061
503477   -36263
422814   -29042
228659   -25039
422542   -24360
308947   -21942
422587   -20866
110134   -14620
308670   -13819
34638    -13697
422682   -10817
310752   -10599
228298    -4993
196647    -2287
399083    -1959
399077    -1861
204369    -1861
399065    -1850
573468    -1848
573474    -1830
380601    -1821
391493    -1812
573462    -1800
204363    -1788
11085     -1779
164458    -1773
289431    -1764
573456    -1761
391481    -1755
11097     -1749
391487    -1729
185749    -1723
399071    -1718
573480    -1699
399059    -1695
483687    -1690
50266     -1685
380607    -1683
11091     -1672
204357    -1670
185743    -1667
196653    -1627
380589    -1622
11103     -1620
391469    -1620
Name: DIFFERENCE, dtype: float64

In [50]:
sorted_df[sorted_df['DIFFERENCE'] == -4993]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,STATID,TIMESTAMP,DIFFERENCE
228298,JFK03,R536,00-00-02,JFK JAMAICA CT1,E,IND,09/06/2016,13:00:00,REGULAR,10,33,JFK JAMAICA CT1E,2016-09-06 13:00:00,-4993


In [51]:
sorted_df[sorted_df['DATE'] == '09/06/2016'][sorted_df['UNIT'] == 'R536'][sorted_df['SCP'] == '00-00-02'].sort_values(['STATID', 'UNIT','SCP', 'TIMESTAMP'])

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,STATID,TIMESTAMP,DIFFERENCE
228295,JFK03,R536,00-00-02,JFK JAMAICA CT1,E,IND,09/06/2016,01:00:00,REGULAR,4898,3333,JFK JAMAICA CT1E,2016-09-06 01:00:00,17
228296,JFK03,R536,00-00-02,JFK JAMAICA CT1,E,IND,09/06/2016,05:00:00,REGULAR,4904,3341,JFK JAMAICA CT1E,2016-09-06 05:00:00,6
228297,JFK03,R536,00-00-02,JFK JAMAICA CT1,E,IND,09/06/2016,09:00:00,REGULAR,5003,3432,JFK JAMAICA CT1E,2016-09-06 09:00:00,99
228298,JFK03,R536,00-00-02,JFK JAMAICA CT1,E,IND,09/06/2016,13:00:00,REGULAR,10,33,JFK JAMAICA CT1E,2016-09-06 13:00:00,-4993
228299,JFK03,R536,00-00-02,JFK JAMAICA CT1,E,IND,09/06/2016,17:00:00,REGULAR,209,196,JFK JAMAICA CT1E,2016-09-06 17:00:00,199
228300,JFK03,R536,00-00-02,JFK JAMAICA CT1,E,IND,09/06/2016,21:00:00,REGULAR,352,324,JFK JAMAICA CT1E,2016-09-06 21:00:00,143


Once again, counter restarted. 

In [52]:
sorted_df.DIFFERENCE.head(50)

228570   -98988
228477   -95614
568904   -87315
228255   -61491
228615   -44919
228388   -40061
503477   -36263
422814   -29042
228659   -25039
422542   -24360
308947   -21942
422587   -20866
110134   -14620
308670   -13819
34638    -13697
422682   -10817
310752   -10599
228298    -4993
196647    -2287
399083    -1959
399077    -1861
204369    -1861
399065    -1850
573468    -1848
573474    -1830
380601    -1821
391493    -1812
573462    -1800
204363    -1788
11085     -1779
164458    -1773
289431    -1764
573456    -1761
391481    -1755
11097     -1749
391487    -1729
185749    -1723
399071    -1718
573480    -1699
399059    -1695
483687    -1690
50266     -1685
380607    -1683
11091     -1672
204357    -1670
185743    -1667
196653    -1627
380589    -1622
11103     -1620
391469    -1620
Name: DIFFERENCE, dtype: float64

In [53]:
sorted_df[sorted_df['DIFFERENCE'] == -2287]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,STATID,TIMESTAMP,DIFFERENCE
196647,A025,R023,01-03-02,34 ST-HERALD SQ,BDFMNQR,BMT,09/07/2016,20:00:00,REGULAR,1557034876,2048728231,34 ST-HERALD SQBDFMNQR,2016-09-07 20:00:00,-2287


In [54]:
sorted_df[sorted_df['DATE'] == '09/07/2016'][sorted_df['UNIT'] == 'R023'][sorted_df['SCP'] == '01-03-02'].sort_values(['STATID', 'UNIT','SCP', 'TIMESTAMP'])

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,STATID,TIMESTAMP,DIFFERENCE
196643,A025,R023,01-03-02,34 ST-HERALD SQ,BDFMNQR,BMT,09/07/2016,00:00:00,REGULAR,1557037983,2048726087,34 ST-HERALD SQBDFMNQR,2016-09-07 00:00:00,-824
196644,A025,R023,01-03-02,34 ST-HERALD SQ,BDFMNQR,BMT,09/07/2016,04:00:00,REGULAR,1557037899,2048726117,34 ST-HERALD SQBDFMNQR,2016-09-07 04:00:00,-84
196645,A025,R023,01-03-02,34 ST-HERALD SQ,BDFMNQR,BMT,09/07/2016,08:00:00,REGULAR,1557037755,2048726376,34 ST-HERALD SQBDFMNQR,2016-09-07 08:00:00,-144
196646,A025,R023,01-03-02,34 ST-HERALD SQ,BDFMNQR,BMT,09/07/2016,12:00:00,REGULAR,1557037163,2048727153,34 ST-HERALD SQBDFMNQR,2016-09-07 12:00:00,-592
196647,A025,R023,01-03-02,34 ST-HERALD SQ,BDFMNQR,BMT,09/07/2016,20:00:00,REGULAR,1557034876,2048728231,34 ST-HERALD SQBDFMNQR,2016-09-07 20:00:00,-2287


For the less extreme negative values, it looks as though the counter is working, however it is counting in reverse.
Because the counter restarts at the difference -4993 and does not at -2287, we will split the data here. Anything less than -4993 will be ignored and the rest will be replaced by their absolute value and included in the count. 

In [55]:
sorted_df = sorted_df[sorted_df['DIFFERENCE'] > -4993]
sorted_df['DIFFERENCE'] = sorted_df['DIFFERENCE'].abs()

In [56]:
sorted_df.DIFFERENCE.tail(10)

308907        193765
458246        261848
112579        534087
358090      16777218
321921      65007495
458290      67108864
358084     100661997
73446      334224743
510927    1145456250
447140    1905096174
Name: DIFFERENCE, dtype: float64

In [57]:
sorted_df = sorted_df[sorted_df['DIFFERENCE'] < 12897]

In [58]:
sorted_df.DIFFERENCE.tail(10)

536583    3022
435259    3027
442743    3062
147790    3115
435253    3167
342223    3207
342229    3252
342237    3281
536158    3366
550602    3837
Name: DIFFERENCE, dtype: float64

In [59]:
clean_df = sorted_df.sort_values(['STATID', 'UNIT','SCP', 'TIMESTAMP'])

In [60]:
clean_df.DIFFERENCE.max()

3837.0

In [70]:
sorted(clean_df.groupby(['STATID'])['DIFFERENCE'].sum(), reverse=True)

[2524667.0,
 2120443.0,
 1583597.0,
 1540467.0,
 1410638.0,
 1283079.0,
 1137239.0,
 1087336.0,
 1035401.0,
 1024708.0,
 956146.0,
 944312.0,
 913029.0,
 867367.0,
 839989.0,
 769928.0,
 729477.0,
 725828.0,
 718493.0,
 715246.0,
 700475.0,
 695195.0,
 597444.0,
 593650.0,
 589648.0,
 579541.0,
 572565.0,
 557362.0,
 550582.0,
 543061.0,
 541001.0,
 540485.0,
 536641.0,
 517337.0,
 514924.0,
 496708.0,
 484878.0,
 483860.0,
 483259.0,
 476939.0,
 472429.0,
 464554.0,
 462114.0,
 460836.0,
 458558.0,
 456986.0,
 451966.0,
 449766.0,
 432956.0,
 428800.0,
 421830.0,
 418142.0,
 406178.0,
 405602.0,
 404281.0,
 402945.0,
 400773.0,
 392845.0,
 389715.0,
 388191.0,
 383650.0,
 382847.0,
 380617.0,
 377360.0,
 376985.0,
 375883.0,
 367796.0,
 366572.0,
 363217.0,
 359756.0,
 359364.0,
 359348.0,
 349607.0,
 347418.0,
 346901.0,
 344141.0,
 340943.0,
 340838.0,
 336461.0,
 334136.0,
 333119.0,
 330307.0,
 328953.0,
 328506.0,
 327593.0,
 327434.0,
 324846.0,
 322960.0,
 322514.0,
 319276.0,


In [62]:
from collections import defaultdict

statid_dict = defaultdict(list)
statids = clean_df['STATID'].unique()

for ID in statids:
    statid_dict['STATID'].append(ID)
    statid_dict['Total_Entries'].append(clean_df[clean_df['STATID'] == ID]['DIFFERENCE'].sum())

In [63]:
statid_df = pd.DataFrame(statid_dict)

In [64]:
statid_df[statid_df['Total_Entries'] == statid_df['Total_Entries'].max()]

,STATID,Total_Entries
310,GRD CNTRL-42 ST4567S,2524667


In [65]:
sorted_statid_df = statid_df.sort_values('Total_Entries', ascending=False)

In [66]:
sorted_statid_df.head()

,STATID,Total_Entries
310,GRD CNTRL-42 ST4567S,2524667
90,34 ST-HERALD SQBDFMNQR,2120443
103,42 ST-PORT AUTHACENQRS1237,1583597
444,TIMES SQ-42 ST1237ACENQRS,1540467
94,34 ST-PENN STAACE,1410638


In [67]:
df[df['STATION'] == 'GRD CNTRL-42 ST'][df['DIFFERENCE'] == 3366.0]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,STATID,TIMESTAMP,DIFFERENCE
536158,R240,R047,00-00-00,GRD CNTRL-42 ST,4567S,IRT,09/16/2016,21:00:00,REGULAR,3327556,6431741,GRD CNTRL-42 ST4567S,2016-09-16 21:00:00,3366


In [68]:
import datetime as dt

weekdays = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
dt_obj = dt.datetime.strptime('09/16/2016', '%m/%d/%Y')
weekdays[dt_obj.weekday()]

'Friday'

The highest amount of entries at a given turnstile was 3366. This was between 5:00 and 9:00 PM on a Friday. 

This process can easily be extended to find out the most dense interval, day, etc. at any given station. 